Сборка сети с предобученным эмбедингом и его тестирование.

In [ ]:
# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer
# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils
# Работа с массивами данных
import numpy as np 
# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential
# Основные слои
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation ,Embedding,SpatialDropout1D,Flatten,Input,LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
# Функции операционной системы
import os
# Регулярные выражения
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences


Загрузка предобученного эмбеддинга glove-wiki-gigaword-50 через API gensim

In [ ]:
import gensim.downloader as api
wv = api.load('glove-wiki-gigaword-50')
wv['beginning'].shape

(50,)

Загрузка датасета

In [ ]:
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=20000)

Оценка датасета средняя длинна и среднеквадратичное отклонение.

In [ ]:
Med=0
std=0
for i in range(len(train_data)):
  Med=Med+len(train_data[i])
Med=Med/len(train_data)
for i in range(len(train_data)):
  std=std+(Med-len(train_data[i]))**2
std=std**0.5
std=std/len(train_data)
print(Med,std)

238.71364 1.1162440026795242


Подготовка текстовых данных

In [ ]:
SequencyWords=100

In [ ]:
train_data2 = pad_sequences(train_data, maxlen=SequencyWords, padding='post', truncating='post') 
test_data2 = pad_sequences(train_data, maxlen=SequencyWords, padding='post', truncating='post')
print(train_data2[0])
print(train_data2[0].shape)
print(test_data2[0].shape)

[    1    14    22    16    43   530   973  1622  1385    65   458  4468
    66  3941     4   173    36   256     5    25   100    43   838   112
    50   670     2     9    35   480   284     5   150     4   172   112
   167     2   336   385    39     4   172  4536  1111    17   546    38
    13   447     4   192    50    16     6   147  2025    19    14    22
     4  1920  4613   469     4    22    71    87    12    16    43   530
    38    76    15    13  1247     4    22    17   515    17    12    16
   626    18 19193     5    62   386    12     8   316     8   106     5
     4  2223  5244    16]
(100,)
(100,)


Получение слоя керас из загруженного эмбединга.

In [ ]:
wv_emb = wv.get_keras_embedding()

Сборка полносвязной модели с предобученным эмбедингом

In [ ]:
input = Input((SequencyWords))
x = wv_emb(input)
x = SpatialDropout1D(0.2)(x)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
Dense(33, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x= Dense(1,activation='sigmoid')(x)

model=Model(input,x)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=1e-4))

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 50)           20000000  
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 100, 50)          0         
 lDropout1D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 5000)              0         
                                                                 
 dense_10 (Dense)            (None, 100)               500100    
                                                                 
 batch_normalization_6 (Batc  (None, 100)              400       
 hNormalization)                                           

In [ ]:
train_data2.shape

(25000, 150)

In [ ]:
model.fit(train_data2, train_labels,validation_data=(test_data2, test_labels), epochs = 30, batch_size = 32)#

Epoch 1/30
782/782 [==============================] - 10s 11ms/step - loss: 0.8591 - accuracy: 0.5054 - val_loss: 0.7128 - val_accuracy: 0.4915
Epoch 2/30
782/782 [==============================] - 8s 11ms/step - loss: 0.8051 - accuracy: 0.5089 - val_loss: 0.7052 - val_accuracy: 0.4980
Epoch 3/30
782/782 [==============================] - 8s 10ms/step - loss: 0.7682 - accuracy: 0.5203 - val_loss: 0.7057 - val_accuracy: 0.4992
Epoch 4/30
782/782 [==============================] - 8s 11ms/step - loss: 0.7457 - accuracy: 0.5267 - val_loss: 0.7038 - val_accuracy: 0.4980
Epoch 5/30
782/782 [==============================] - 8s 11ms/step - loss: 0.7250 - accuracy: 0.5383 - val_loss: 0.7040 - val_accuracy: 0.4953
Epoch 6/30
782/782 [==============================] - 8s 10ms/step - loss: 0.7153 - accuracy: 0.5441 - val_loss: 0.7034 - val_accuracy: 0.4971
Epoch 7/30
782/782 [==============================] - 8s 11ms/step - loss: 0.7021 - accuracy: 0.5514 - val_loss: 0.7060 - val_accuracy: 0.496

Сборка LSTM модели с предобученным эмбедингом

In [ ]:
input = Input((SequencyWords))
x = wv_emb(input)
#x = SpatialDropout1D(0.2)(x)
x = LSTM(200)(x)
#x = Dropout(0.2)(x)
x = Dense(33, activation="relu")(x)
x = BatchNormalization()(x)
#x = Dropout(0.1)(x)
x= Dense(1,activation='sigmoid')(x)
model=Model(input,x)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=1e-4))
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 50)           20000000  
                                                                 
 lstm_2 (LSTM)               (None, 200)               200800    
                                                                 
 dense_8 (Dense)             (None, 33)                6633      
                                                                 
 batch_normalization_5 (Batc  (None, 33)               132       
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 1)                 34        
                                                           

In [ ]:
model.fit(train_data2, train_labels,validation_data=(test_data2, test_labels), epochs = 10, batch_size = 32)#

Epoch 1/10
782/782 [==============================] - 211s 266ms/step - loss: 0.6995 - accuracy: 0.5312 - val_loss: 0.7135 - val_accuracy: 0.5001
Epoch 2/10
782/782 [==============================] - 208s 267ms/step - loss: 0.6856 - accuracy: 0.5519 - val_loss: 0.8094 - val_accuracy: 0.4972
Epoch 3/10
782/782 [==============================] - 209s 267ms/step - loss: 0.6677 - accuracy: 0.5902 - val_loss: 0.7523 - val_accuracy: 0.5018
Epoch 4/10
782/782 [==============================] - 210s 268ms/step - loss: 0.6512 - accuracy: 0.6176 - val_loss: 0.7649 - val_accuracy: 0.4953
Epoch 5/10
782/782 [==============================] - 208s 266ms/step - loss: 0.6277 - accuracy: 0.6426 - val_loss: 0.8329 - val_accuracy: 0.4949
Epoch 6/10
782/782 [==============================] - 207s 265ms/step - loss: 0.6044 - accuracy: 0.6719 - val_loss: 1.1893 - val_accuracy: 0.4989
Epoch 7/10
782/782 [==============================] - 206s 264ms/step - loss: 0.5872 - accuracy: 0.6841 - val_loss: 0.9673 -

У LSTM точность повыше получается.

Дальше эксперимент по восстановлению фраз

In [ ]:
word_to_id = imdb.get_word_index()

In [ ]:
INDEX_FROM=3   # word index offset
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3
id_to_word = {value:key for key,value in word_to_id.items()}
train_data_txt=[]
test_data_txt=[]
for i in range(len(train_data2)):
  train_data_txt.append(' '.join(id_to_word[id] for id in train_data2[i]))
for i in range(len(test_data2)):
  test_data_txt.append(' '.join(id_to_word[id] for id in test_data2[i]))

print(train_data_txt[4])
print(test_data_txt[5])

"<START> begins better than it ends funny that the russian submarine crew <UNK> all other actors it's like those scenes where documentary shots br br spoiler part the message <UNK> was contrary to the whole story it just does not mesh br br <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA